In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('2022Games.csv')

print(df)

     GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0       2022-03-12  22101005            Final    1610612748       1610612750   
1       2022-03-12  22101006            Final    1610612741       1610612739   
2       2022-03-12  22101007            Final    1610612759       1610612754   
3       2022-03-12  22101008            Final    1610612744       1610612749   
4       2022-03-12  22101009            Final    1610612743       1610612761   
...            ...       ...              ...           ...              ...   
1071    2021-10-04  12100007            Final    1610612759       1610612762   
1072    2021-10-04  12100008            Final    1610612757       1610612744   
1073    2021-10-04  12100009            Final    1610612758       1610612756   
1074    2021-10-04  12100010            Final    1610612746       1610612743   
1075    2021-10-03  12100001            Final    1610612747       1610612751   

      SEASON  TEAM_ID_home  PTS_home  F

##Logistic Regression

In [ ]:
y = df['HOME_TEAM_WINS']
print(y)

0       0
1       1
2       0
3       1
4       0
       ..
1071    1
1072    0
1073    1
1074    1
1075    0
Name: HOME_TEAM_WINS, Length: 1076, dtype: int64


In [ ]:
Z = df.drop(['PTS_home','SEASON','TEAM_ID_home','HOME_TEAM_WINS','PTS_away','TEAM_ID_away'], axis =1)
Z['FG_Diff_PT'] = 100* (Z['FG_PCT_home'] - Z['FG_PCT_away'])
Z['FT_Diff_PT'] = 100*(Z['FT_PCT_home'] - Z['FT_PCT_away'])
Z['FG3_Diff_PT'] = 100*(Z['FG3_PCT_home'] - Z['FG3_PCT_away'])
Z['AST_Diff'] = Z['AST_home'] - Z['AST_away']
Z['REB_Diff'] = Z['REB_home'] - Z['REB_away']
X = Z[['FG_Diff_PT','FT_Diff_PT','FG3_Diff_PT','AST_Diff','REB_Diff']]
X

,FG_Diff_PT,FT_Diff_PT,FG3_Diff_PT,AST_Diff,REB_Diff
0,-2.4,-11.5,-2.4,2.0,7.0
1,2.4,10.9,22.1,1.0,6.0
2,-7.7,-18.7,-6.5,5.0,5.0
3,7.1,23.7,1.4,6.0,16.0
4,8.0,-1.0,2.0,4.0,-11.0
...,...,...,...,...,...
1071,14.8,16.0,12.0,8.0,13.0
1072,2.6,0.8,7.3,-11.0,6.0
1073,0.6,-16.2,2.6,-2.0,13.0
1074,0.2,13.9,6.3,6.0,7.0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_lr_train_pred = lr.predict(X_train)
y_lr_test_pred = lr.predict(X_test)

In [ ]:
lr.score(X, y)

0.8661710037174721

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
lr_train_mse = mean_squared_error(y_train,y_lr_train_pred)
lr_train_r2 = r2_score(y_train, y_lr_train_pred)

lr_test_mse = mean_squared_error(y_test, y_lr_test_pred)
lr_test_r2 = r2_score(y_test, y_lr_test_pred)

print("LR Test R2:", lr_test_r2)
print("LR Test MSE:", lr_test_mse)
print("LR Train R2:", lr_train_r2)
print("LR Test MSE:", lr_train_mse)

LR Test R2: 0.4692965344639455
LR Test MSE: 0.13211850034724948
LR Train R2: 0.5152044405632572
LR Test MSE: 0.11942646261105569


##Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [ ]:
y = df['HOME_TEAM_WINS']
print(y)

In [ ]:
Z = df.drop(['PTS_home','SEASON','TEAM_ID_home','HOME_TEAM_WINS','PTS_away','TEAM_ID_away'], axis =1)
Z['GAME_DATE_EST'] = 100* (Z['FG_PCT_home'] - Z['FG_PCT_away'])
Z['GAME_ID'] = 100*(Z['FT_PCT_home'] - Z['FT_PCT_away'])
Z['GAME_STATUS_TEXT'] = 100*(Z['FG3_PCT_home'] - Z['FG3_PCT_away'])
Z['HOME_TEAM_ID'] = Z['AST_home'] - Z['AST_away']
Z['VISITOR_TEAM_ID'] = Z['REB_home'] - Z['REB_away']
Z = Z.drop(['REB_away','AST_away','FG3_PCT_away','FT_PCT_away','FG_PCT_away','REB_home','AST_home','FG3_PCT_home','FT_PCT_home','FG_PCT_home'], axis =1)
Z

In [ ]:
print(f'Z : {Z.shape}')

In [ ]:
from sklearn.model_selection import train_test_split

Z_train, Z_test, y_train, y_test = train_test_split(Z, y, test_size=0.2, random_state=101)

### Hyperparameters

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [2,4]
min_samples_split = [2,5]
min_samples_leaf = [1,2]
bootstrap = [True, False]

In [ ]:
param_grid = {'n_estimators':n_estimators,
              'max_features':max_features,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf,
              'bootstrap':bootstrap
              }
print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [ ]:
rf_Model = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid,  cv = 3, verbose=2, n_jobs = 4)

In [ ]:
rf_Grid.fit(Z_train, y_train)

Fitting 3 folds for each of 320 candidates, totalling 960 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72,
                                          80]},
             verbose=2)

In [ ]:
rf_Grid.best_params_

{'bootstrap': False,
 'max_depth': 4,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 48}

In [ ]:
print(f'Train Accuracy - : {rf_Grid.score(Z_train, y_train):.3f}')
print(f'Test Accuracy - : {rf_Grid.score(Z_test, y_test):.3f}')

Train Accuracy - : 0.901
Test Accuracy - : 0.819


## Test on Past Years

In [ ]:
df2 = pd.read_csv('2021Games.csv')

print(df2)

In [ ]:
y2 = df2['HOME_TEAM_WINS']
print(y)

In [ ]:
Z2 = df2.drop(['PTS_home','SEASON','TEAM_ID_home','HOME_TEAM_WINS','PTS_away','TEAM_ID_away'], axis =1)
Z2['GAME_DATE_EST'] = 100* (Z2['FG_PCT_home'] - Z2['FG_PCT_away'])
Z2['GAME_ID'] = 100*(Z2['FT_PCT_home'] - Z2['FT_PCT_away'])
Z2['GAME_STATUS_TEXT'] = 100*(Z2['FG3_PCT_home'] - Z2['FG3_PCT_away'])
Z2['HOME_TEAM_ID'] = Z2['AST_home'] - Z2['AST_away']
Z2['VISITOR_TEAM_ID'] = Z2['REB_home'] - Z2['REB_away']
Z2 = Z2.drop(['REB_away','AST_away','FG3_PCT_away','FT_PCT_away','FG_PCT_away','REB_home','AST_home','FG3_PCT_home','FT_PCT_home','FG_PCT_home'], axis =1)
Z2

In [ ]:
from sklearn.model_selection import train_test_split

Z2_train, Z2_test, y2_train, y2_test = train_test_split(Z2, y2, test_size=0.2, random_state=101)

In [ ]:
print(f'Train Accuracy - : {rf_Grid.score(Z2_train, y2_train):.3f}')
print(f'Test Accuracy - : {rf_Grid.score(Z2_test, y2_test):.3f}')

Train Accuracy - : 0.831
Test Accuracy - : 0.838
